In [ ]:
# %matplotlib inline
# %load_ext tensorboard

In [ ]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = str(0)

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
import sys
import tensorflow as tf
from tensorflow import keras  # tf.keras
import time

In [ ]:
print("python", sys.version)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

In [ ]:
assert sys.version_info >= (3, 5) # Python ≥3.5 required
assert tf.__version__ >= "2.0"    # TensorFlow ≥2.0 required

In [ ]:
from Mike_NB_tools import *
from Transformer import *

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            # logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            # print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

In [ ]:
from datetime import datetime, timedelta
import sys 
sys.path.append('..')
import json

In [ ]:
# ============================  Define training parameters ===========================

Nx = 600
Ny = 5
Ns = 5
BatchSize = 64

CandleFile = "18-01-01-00-00-23-05-20-20-23-5m"
SmallSigma = 1
LargeSigma = 30
eFreeNoLog = True

nFiles_t = 70
nFiles_v = 30
n_readers = 20
shuffle_batch = 200
nPrefetch = 3

dir_datasets = "/mnt/data/Trading/Datasets"
dir_candles = "/mnt/data/Trading/Candles"

chosen_markets_x = [
                    'DOTUSDT', 'BTCUSDT', 'ETHUSDT', 'LINKUSDT', 'ETCUSDT', 'XLMUSDT', 'STPTUSDT',   # cluster 2, with over 50% true candles, except 'DOTUSDT'
                    'EOSUSDT', 'BNTUSDT', # cluster 1, with over 50% true candles.
                    'ANKRUSDT', # cluster 0, with over 50% true candles.
                    'BNBUSDT', 'AVAXUSDT', 'COMPUSDT', 'BALUSDT', 'XRPUSDT', '1INCHUSDT'  # etc
]
chosen_fields_x = ['ClosePrice'] #, 'BaseVolume']
chosen_markets_y = [
                    'BTCUSDT', 'ETHUSDT', 'LINKUSDT', 'ETCUSDT', 'XLMUSDT',   # cluster 2, with over 50% true candles, except 'DOTUSDT'
                    'BNBUSDT', 'AVAXUSDT', 'BALUSDT', 'XRPUSDT', '1INCHUSDT'  # etc
]
chosen_fields_y = ['ClosePrice']

Standardization = False
Time_into_X = True
Time_into_Y = False
Transformer = False
Reuse_files = False

Epochs_Initial = 50


In [ ]:
#==================== Load candle data into 'table' with shape of (time, markets, 10 fields) ====================
Candles = np.load( os.path.join( dir_candles, "table-" + CandleFile + ".npy") )
Candles = np.swapaxes(Candles, 0, 1)
print("Candles: {}".format(Candles.shape))

In [ ]:
market = 5
Show_Price_Volume_10(Candles[:, market, :], 1, 1, 5000)

In [ ]:
Event_Free_Learning_Scheme_10(Candles[:, market, :], 3, 30, 5000)

In [ ]:
#==================== Delete 7 candle fields from 'Candles'. ====================
# Candles.shape becomes (time, markets, ['ClosePrice', 'BaseVolume', 'BuyerBaseVolume'] )

CandleMarks = Candles[:, :, 9] # keep it for later use
Candles = np.delete(Candles, [0, 1, 2, 5, 6, 8, 9], axis = 2) # delete Open, High, Low, qVolume, #Trades, bQVolume, CandleMarks

table_markets = []
with open( os.path.join( dir_candles, "reports-" + CandleFile + ".json"), "r") as f:
    reports = json.loads(f.read())
print(reports[:2])

markets = [ s[0: s.find(':')] for s in reports if 'Success' in s ]
assert Candles.shape[1] == len(markets)
print(Candles.shape, len(markets), markets[:2])

In [ ]:
#==================== Restore timestamps_abs. ====================

start = datetime( 2000+int(CandleFile[0:2]), int(CandleFile[3:5]), int(CandleFile[6:8]), int(CandleFile[9:11]), int(CandleFile[12:14]) )
start_ts = round(datetime.timestamp(start))
interval = CandleFile[ CandleFile.find('-', len(CandleFile) - 4) + 1 : ]
interval_s = round(intervalToMilliseconds(interval) / 1000)
timestamps_abs = np.array( range(start_ts, start_ts + Candles.shape[0] * interval_s, interval_s), dtype=int)
assert timestamps_abs.shape[0] == Candles.shape[0]
print(start_ts, interval_s, timestamps_abs.shape, timestamps_abs[:3])

In [ ]:
parse_csv_line_to_tensors(b'0., 1., 2., 3., 4., 5., 6., 7., 222., 333., 8., 9., 10., 11., 444., 555.', 2, 4, 2, 2, True, True, 1) # nx, size_x, ny, size_y, time_x, time_y, sixe_time

In [ ]:
parse_csv_line_to_tensors(b'0., 1., 2., 3., 4., 5., 6., 7., 222., 333., 8., 9., 10., 11.', 2, 4, 2, 2, True, False, 1) # nx, size_x, ny, size_y, time_x, time_y, sixe_time

In [ ]:
parse_csv_line_to_tensors(b'0., 1., 2., 3., 4., 5., 6., 7., 8., 9., 10., 11.', 2, 4, 2, 2, False, False, 1) # nx, size_x, ny, size_y, time_x, time_y, sixe_time

In [ ]:
parse_csv_line_to_tensors_for_transformer(b'0., 1., 2., 3., 4., 5., 6., 7., 222., 333., 8., 9., 10., 11., 444., 555.', 2, 4, 2, 2, True, True, 1) # nx, size_x, ny, size_y, time_x, time_y, sixe_time

In [ ]:
parse_csv_line_to_tensors_for_transformer(b'0., 1., 2., 3., 4., 5., 6., 7., 222., 333., 8., 9., 10., 11.', 2, 4, 2, 2, True, False, 1) # nx, size_x, ny, size_y, time_x, time_y, sixe_time

In [ ]:
parse_csv_line_to_tensors_for_transformer(b'0., 1., 2., 3., 4., 5., 6., 7., 8., 9., 10., 11.', 2, 4, 2, 2, False, False, 1) # nx, size_x, ny, size_y, time_x, time_y, sixe_time

In [ ]:
# defind test data

n_times = 1000; n_markets = 2; n_fields = 2
data = [ [ [ time * n_markets * n_fields + market * n_fields + field for field in range(n_fields) ] for market in range(n_markets) ] for time in range(n_times)]
data = np.array(data, dtype=float)
times_test = np.array( range(data.shape[0]) ) + 100000
print(data.shape, times_test.shape)   # time, market, field
print(data[:2, :, :])

In [ ]:
print(data.shape, times_test.shape)   # time, market, field

In [ ]:
nx_test = 2
ny_test = 2
ns_test = 10
batchSize = 2

sample_anchors = range(0, data.shape[0] - nx_test - ny_test, ns_test)
print(data.shape[0], len(sample_anchors), sample_anchors)

x_indices = ( (0, 1), (0, 1) )    # (market, field)
y_indices = ( (0,), (0, 1) )    # (market, field)
print(data[0:2][:, x_indices[0]][:, :, x_indices[1]])
print(data[2:4][:, y_indices[0]][:, :, y_indices[1]])

size_x = get_timepoint_size(x_indices)
size_y = get_timepoint_size(y_indices)
size_time = 1
print(size_x, size_y, size_time)


In [ ]:
name_plus = CandleFile+'_o'
name_prefix = os.path.join(dir_datasets, name_plus)

reuse_files = False

if reuse_files:
    import re
    filenames = [ os.path.join(dir_datasets, x) for x in os.listdir(dir_datasets) if re.match(name_plus, x)]
else:
    os.system("rm {}/*{}*".format(dir_datasets, name_plus))
    filenames = divide_to_multiple_csv_files(data, True, True, times_test, sample_anchors, name_prefix, nx_test, x_indices, ny_test, y_indices, header=None, n_parts=10)

print(filenames)

In [ ]:
filename_dataset = tf.data.Dataset.list_files(filenames, shuffle=None) # no way to prevent shuffle.
print(filename_dataset.cardinality().numpy())
for element in filename_dataset:
    print(element.numpy())

In [ ]:
ds = tf.data.TextLineDataset(filenames[0])
for line in ds.take(20):
    print(line.numpy())

In [ ]:
n_readers = 5
dataset = filename_dataset.interleave(
    lambda filename: tf.data.TextLineDataset(filename),
    cycle_length=n_readers, num_parallel_calls=tf.data.AUTOTUNE) # no way to prevent shuffle?

for line in dataset.take(15):
    print(line.numpy())

In [ ]:
dataset = csv_reader_to_dataset(filenames, nx_test, size_x, ny_test, size_y, True, True, size_time,
                             n_parse_threads=5, batch_size=batchSize, shuffle_buffer_size=1000, n_readers=5)

In [ ]:
for element in dataset:
    print(element)
    break

# should print: (None, nx_test, size_x + size_time), (None, ny_test * size_y)

In [ ]:
# Check elements: NaN, -inf, +inf

assert (~np.isfinite(Candles)).any() == False

In [ ]:
nLatest = 500
P, maP, logP, log_maP, event, eventFree = Get_eFree(Candles[:, 0, 0], 1, 30, nLatest)
assert maP.shape[0] == nLatest; assert logP.shape[0] == nLatest; assert log_maP.shape[0] == nLatest; assert event.shape[0] == nLatest; assert eventFree.shape[0] == nLatest


In [ ]:
#==================== Define 'get_eFree_with_plot' ====================

def get_eFree_with_plot(market, field, feature, smallSigma, largeSigma, nLatest, noPlot = True, noLog = False):
    if noLog:
        P, maP, _, _, event, eventFree = Get_eFree_noLog(feature, smallSigma, largeSigma, nLatest)
        series = [ [maP, "maP", "g"], [event, "event", "c"],  [eventFree, "e.Free", "brown"] ] #, [P, "raw feature", "r"] ]
        if not noPlot:
            PoltNormalized("Event-free (brown) {} on {}".format(field, market), series)
        return P, maP, _, _, event, eventFree
    else:
        P, maP, logP, log_maP, event, eventFree = Get_eFree(feature, smallSigma, largeSigma, nLatest)
        series = [ [maP, "maP", "g"], [logP, "logP" ,"m"], [log_maP, "log.maP", "b"], [event, "event", "c"],  [eventFree, "e.Free", "brown"] ] #, [P, "raw feature", "r"] ]
        if not noPlot:
            PoltNormalized("Event-free (brown) {} on {}".format(field, market), series)
        return P, maP, logP, log_maP, event, eventFree

In [ ]:
clusters = [
    ['APT', 'SUI', 'DYDX', 'ANKR', 'AUDIO', 'SKL'],
    ['EOS', 'AAVE', 'FLOW', 'CRV', 'COMP', 'SLP', 'MBOX', 'BNT', 'SPELL', 'AERGO', 'BAKE'],
    ['DOT', 'BTC', 'ETH', 'WBTC', 'LINK', 'ETC', 'XLM', 'TWT', 'SFP', 'STPT', 'STEEM', 'POWR'],
]

for c in range(len(clusters)):
    cluster = clusters[c]
    cluster = [ markets.index(m + 'USDT') for m in cluster ]
    check = [ (markets[m], 100 - round(np.argmax(Candles[:, m, 0]>0) / Candles.shape[0] * 100)) for m in cluster ]
    print(check)

In [ ]:
#==================== Define Data ====================

Data = Candles[:, :, :]   # (time:, all markets, 20 fields)

In [ ]:
check = np.array([ np.argmax(Data[:, m, 0]>0) / Data.shape[0] * 100 for m in range(len(markets)) ])
permute = np.argsort(check)
marketrank = [ (markets[m], 100 - round(np.argmax(Data[:, m, 0]>0) / Data.shape[0] * 100)) for m in permute ]
# marketrank = [ markets[m] for m in permute ]

batch = 10
for i in range(0, len(markets), batch):
    print(marketrank[i: i+batch])

In [ ]:
#==================== Select markets and fields ====================

enFields = ['ClosePrice', 'BaseVolume', 'BuyerBaseVolume']

# dot, 1inch, btc, eth, matic, bnb, ada, sol, ltc, avax, wbtc, link, arb, ape, aave, crv, sui, op, gmx, agix, bal, comp, gmt, joe, stg

chosen_markets_x = tuple([ markets.index(elem) for elem in chosen_markets_x ])
chosen_markets_x = tuple(list(set(chosen_markets_x)))

chosen_fields_x = tuple( [ enFields.index(elem) for elem in chosen_fields_x ] )
chosen_fields_x = tuple(list(set(chosen_fields_x)))
x_indices = ( chosen_markets_x, chosen_fields_x )
print(x_indices)

chosen_markets_y = tuple([ markets.index(elem) for elem in chosen_markets_y ])
chosen_markets_y = tuple(list(set(chosen_markets_y)))

chosen_fields_y = tuple( [ enFields.index(elem) for elem in chosen_fields_y ] )
chosen_fields_y = tuple(list(set(chosen_fields_y)))
y_indices = ( chosen_markets_y, chosen_fields_y )
print(y_indices)

size_x = get_timepoint_size(x_indices)
size_y = get_timepoint_size(y_indices)
print(size_x, size_y)

chosen_markets = tuple(list(set(chosen_markets_x + chosen_markets_y)))
chosen_fields = tuple(list(set(chosen_fields_x + chosen_fields_y)))
print(chosen_markets, chosen_fields)

In [ ]:
#==================== Generate event-free data into Data ====================
# Data loses heading items.
# Do it before: Permute Data in time

alpha = 3; beta = 3 # beta is used in 'get_eFree_with_plot'. Ugly coupling.
event_free_data_loss = 3 * ( alpha * SmallSigma + LargeSigma)
eFree = np.zeros( (Data.shape[0] - event_free_data_loss, len(chosen_markets), len(chosen_fields)), dtype = np.float32 )

for market in chosen_markets:
    for field in chosen_fields:
        sSigma = SmallSigma
        if enFields[field] == 'BaseVolume': sSigma = SmallSigma * alpha
        P, maP, logP, log_maP, event, eventFree = \
        get_eFree_with_plot(markets[market], enFields[field], Data[:, market, field], sSigma, 
                            LargeSigma, Data.shape[0] - event_free_data_loss, noPlot=False, noLog=eFreeNoLog)
        Data[event_free_data_loss:, market, field] = eventFree

Data = Data[event_free_data_loss: ]

print(Data.shape)

In [ ]:
#==================== Define time features, to augment Data with ====================

sigma = np.power(2.0, -0.2)
hourly = np.sin( 2 * np.pi / (60*60) * timestamps_abs ) / sigma
daily = np.sin( 2 * np.pi / (60*60*24) * timestamps_abs ) / sigma
weekly = np.sin( 2 * np.pi / (60*60*24*7) * timestamps_abs ) / sigma
yearly = np.sin( 2 * np.pi / (60*60*24*365) * timestamps_abs ) / sigma

Time = np.stack([hourly, daily, weekly, yearly], axis=1)
size_time = Time.shape[1]

Time = Time[event_free_data_loss: ]
assert Data.shape[0] == Time.shape[0]
print(Candles.shape, Time.shape)

In [ ]:
#==================== Standardize Data on chosen markets and fields ====================

Standard = []

if Standardization:
    for market in chosen_markets:
        for field in chosen_fields:
            nzPs = np.where( Data[:, market, field] != 0.0 ) [0]
            mu = np.average(Data[nzPs, market, field])
            sigma = np.std(Data[nzPs, market, field])
            standard = (Data[nzPs, market, field] - mu) / (sigma + 1e-15)
            Standard.append( (market, field, mu, sigma) )
            Data[nzPs, market, field] = standard

    Standard = np.array(Standard)

In [ ]:
fig = plt.figure(figsize=(16,3))
ax = fig.add_subplot(111)
ax.set_title("Features are custom-standardized" if Standardization else "Features are not standardized")
for market in chosen_markets:
    for field in chosen_fields:
        ax.plot(Data[:, market, field], label = "{} @ {}".format(enFields[field], markets[market][:-4])) # -4: 'USDT'
ax.legend(loc = 'upper left')
plt.show()

In [ ]:
#==================== Define input sequence and output sequence ====================

sample_anchors = np.array(range(0, Data.shape[0] - Nx - Ny + 1, Ns))
print(Data.shape[0], len(sample_anchors), sample_anchors, sample_anchors[-1])
print(Data.shape[0], sample_anchors[ -1 ], sample_anchors[ -1 ] + Nx + Ny, sample_anchors[ -1 ] + Ns, sample_anchors[ -1 ] + Ns + Nx + Ny)

for _ in range(10):
    permute = np.random.permutation(sample_anchors.shape[0])
    sample_anchors = sample_anchors[permute]

from sklearn.model_selection import train_test_split
sample_anchores_t, sample_anchores_v = train_test_split(sample_anchors, test_size=0.30, random_state=42)
print(sample_anchores_t.shape, sample_anchores_v.shape)

sample_anchores_t = tuple(sample_anchores_t)
sample_anchores_v = tuple(sample_anchores_v)

In [ ]:
#==================== Create train/valid datasets ====================

name_plus_t = CandleFile+'_t'
name_plus_v = CandleFile+'_v'
name_prefix_t = os.path.join(dir_datasets, name_plus_t)
name_prefix_v = os.path.join(dir_datasets, name_plus_v)

reuse_files = Reuse_files #------------------------------------------------------------------------------------------------------- 

if reuse_files:
    import re
    filenames_train = [ os.path.join(dir_datasets, x) for x in os.listdir(dir_datasets) if re.match(name_plus_t, x)]
    filenames_valid = [ os.path.join(dir_datasets, x) for x in os.listdir(dir_datasets) if re.match(name_plus_v, x)]
else:
    os.system("rm {}/*{}*".format(dir_datasets, name_plus_t))
    os.system("rm {}/*{}*".format(dir_datasets, name_plus_v))
    filenames_train = divide_to_multiple_csv_files(Data, Time_into_X, Time_into_Y, Time, sample_anchores_t, name_prefix_t, Nx, x_indices, Ny, y_indices, header=None, n_parts=nFiles_t)
    filenames_valid = divide_to_multiple_csv_files(Data, Time_into_X, Time_into_Y, Time, sample_anchores_v, name_prefix_v, Nx, x_indices, Ny, y_indices, header=None, n_parts=nFiles_v)

# sample_anchores are already shuffled. But we need to shuffle datasets again, because it will reshuffle at every epoch.
Dataset_train = csv_reader_to_dataset(filenames_train, Nx, size_x, Ny, size_y, Time_into_X, Time_into_Y, size_time,
                             n_parse_threads=5, batch_size=BatchSize, shuffle_buffer_size=BatchSize*shuffle_batch, n_readers=n_readers, transformer=Transformer)
Dataset_train = Dataset_train.prefetch(nPrefetch)

Dataset_valid = csv_reader_to_dataset(filenames_valid, Nx, size_x, Ny, size_y, Time_into_X, Time_into_Y, size_time,
                             n_parse_threads=5, batch_size=BatchSize, shuffle_buffer_size=BatchSize*shuffle_batch, n_readers=n_readers, transformer=Transformer)
Dataset_valid = Dataset_valid.prefetch(nPrefetch)

In [ ]:
for element in Dataset_train:
    print(element)
    break

In [ ]:
#==================== Define RegularizedLSTM ====================

from functools import partial
RegularizedLSTM = partial(keras.layers.LSTM,
                          return_sequences=True,
                          kernel_regularizer=keras.regularizers.l2(1e-4),
                          recurrent_regularizer=keras.regularizers.l2(1e-4))

In [ ]:
STEPS_PER_EPOCH = len(sample_anchores_t) // BatchSize

lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
  0.001, # default: 0.001 
  decay_steps=STEPS_PER_EPOCH*1000,
  decay_rate=1,
  staircase=False)

def get_optimizer():
  return tf.keras.optimizers.Adam(lr_schedule)

step = np.linspace(0,100000)
lr = lr_schedule(step)
plt.figure(figsize = (8,6))
plt.plot(step/STEPS_PER_EPOCH, lr)
plt.ylim([0,max(plt.ylim())])
plt.xlabel('Epoch')
_ = plt.ylabel('Learning Rate')

In [ ]:
def get_callbacks(name):
  return [
    tfdocs.modeling.EpochDots(),
    tf.keras.callbacks.EarlyStopping(monitor='val_binary_crossentropy', patience=200),
    tf.keras.callbacks.TensorBoard(logdir/name),
  ]

In [ ]:
#==================== Define build_model ====================


def build_model(input_dim, output_size, allow_cudnn_kernel=True):

    units = max(input_dim, output_size)

    # CuDNN is only available at the layer level, and not at the cell level.
    # This means `LSTM(units)` will use the CuDNN kernel,
    # while RNN(LSTMCell(units)) will run on non-CuDNN kernel.
    if allow_cudnn_kernel:
        # The LSTM layer with default options uses CuDNN.

        inputs = keras.Input( shape = (None, input_dim), name = "candles" )
        x = keras.layers.BatchNormalization()(inputs)       
        x = RegularizedLSTM(units, kernel_initializer="lecun_normal")(x)
        x = keras.layers.BatchNormalization()(x)
        x = RegularizedLSTM(units, kernel_initializer="lecun_normal")(x)
        x = keras.layers.BatchNormalization()(x)
        x = RegularizedLSTM(units, kernel_initializer="lecun_normal")(x)
        x = keras.layers.BatchNormalization()(x)
        x = RegularizedLSTM(units, kernel_initializer="lecun_normal")(x)
        x = keras.layers.BatchNormalization()(x)
        x = RegularizedLSTM(units, kernel_initializer="lecun_normal")(x)
        x = keras.layers.BatchNormalization()(x)
        x = RegularizedLSTM(units, kernel_initializer="lecun_normal")(x)
        x = keras.layers.BatchNormalization()(x)
        x = RegularizedLSTM(units, kernel_initializer="lecun_normal")(x)
        x = keras.layers.BatchNormalization()(x)
        x = RegularizedLSTM(units, kernel_initializer="lecun_normal")(x)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.LSTM(units, kernel_initializer="lecun_normal")(x)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dense(units, activation="selu", kernel_initializer="lecun_normal")(x)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dense(units, activation="selu", kernel_initializer="lecun_normal")(x)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dense(units, activation="selu", kernel_initializer="lecun_normal")(x)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dense(units, activation="selu", kernel_initializer="lecun_normal")(x)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dense(units, activation="selu", kernel_initializer="lecun_normal")(x)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dense(units, activation="selu", kernel_initializer="lecun_normal")(x)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dense(units, activation="selu", kernel_initializer="lecun_normal")(x)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dense(units, activation="selu", kernel_initializer="lecun_normal")(x)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dense(units, activation="selu", kernel_initializer="lecun_normal")(x)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dense(units, activation="selu", kernel_initializer="lecun_normal")(x)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dense(units, activation="selu", kernel_initializer="lecun_normal")(x)
        x = keras.layers.BatchNormalization()(x)

        outputs = keras.layers.Dense(output_size, activation="selu", kernel_initializer="lecun_normal")(x)

        model = keras.Model( 
            inputs = inputs, 
            outputs = outputs,
            name = "LSTMDense_model"
        )

        model.compile(
            # loss=keras.losses.MeanAbsoluteError(name='loss'),
            loss=MaskedMSE(),
            optimizer=keras.optimizers.Adam(
                learning_rate=0.0001,  # def lr = 0.001
                beta_1=0.9,
                beta_2=0.999, 
                epsilon=1e-07
            ),
            metrics=keras.metrics.MSE,
        )

    return model

In [ ]:
#==================== Build model ====================

model = build_model(size_x + (size_time if Time_into_X else 0), Ny * size_y, allow_cudnn_kernel=True)

model.summary()

In [ ]:
#==================== Fit model ====================

history = model.fit(Dataset_train, validation_data=Dataset_valid, epochs=Epochs_Initial)
# history_2= model.fit(Dataset_train, validation_data=Dataset_valid, epochs=10, initial epoch=history_1.epoch(-1) )

In [ ]:
plot_history(history)

In [ ]:
# history_2= model.fit(Dataset_train, validation_data=Dataset_valid, epochs=10, initial epoch=history_1.epoch(-1) )

In [ ]:
plot_history(history)

In [ ]:
history = model.fit(Dataset_train, validation_data=Dataset_valid, epochs=50)

In [ ]:
plot_history(history)

In [ ]:
history = model.fit(Dataset_train, validation_data=Dataset_valid, epochs=50)

In [ ]:
plot_history(history)

In [ ]:
history = model.fit(Dataset_train, validation_data=Dataset_valid, epochs=50)

In [ ]:
plot_history(history)

In [ ]:
history = model.fit(Dataset_train, validation_data=Dataset_valid, epochs=50)

In [ ]:
plot_history(history)

In [ ]:
history = model.fit(Dataset_train, validation_data=Dataset_valid, epochs=50)

In [ ]:
plot_history(history)

In [ ]:
history = model.fit(Dataset_train, validation_data=Dataset_valid, epochs=50)

In [ ]:
plot_history(history)

In [ ]:
history = model.fit(Dataset_train, validation_data=Dataset_valid, epochs=50)

In [ ]:
plot_history(history)